# Deep Learning con Python

## Redes Neuronales Recurrentes (RNN)

En esta parte del taller vamos a estudiar las **redes recurrentes** aplicadas a la predicción horario de temperatura. Se implementan y comparan tres arquitecturas:

- RNN simple (`SimpleRNN`)
- LSTM (`LSTM`)
- GRU (`GRU`)

Este notebook  incluye:
- Obtención de datos horarios desde la API pública de Open-Meteo (función proporcionada por el enunciado adaptada a una clase `TemperatureFetcher`).
- Preprocesado (resampling, escalado, creación de secuencias).
- Implementación de modelos Keras para predicción 1-step y forecasting multi-step (modo autoregresivo vs usar los datos reales como entradas).
- Experimentos con diferentes horizontes de predicción y métricas de evaluación.


---


## 0) Predicción de series temporales ¿Qué es eso?

In [ ]:
display(Image(url="https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/main/img/ml_forecasting_multi-step.gif",width=900, height=300))

Para ello vamos a seguir el mismo flujo de trabajo que ya usamos a la hora de desarrollar nuestro MLP

In [ ]:
display(Image(url="https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/main/img/ml_pipeline.jpg",width=800, height=300))

---

## 1) Librerías principales

In [ ]:
import datetime
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, GRU
from tensorflow.keras.callbacks import EarlyStopping

from IPython.display import Image, display

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

---

## 2) Obtener la serie temporal sobre la que vamos a trabajar

En esta sección definimos una pequeña clase  `TemperatureFetcher` cuya responsabilidad es encapsular la descarga de un histórico de datos de temperatura horaria desde la API pública de `Open‑Meteo`.


#### Qué hace exactamente esta clase


- *Constructor (`__init__`)*: guarda las coordenadas (`latitude`, `longitude`) del lugar para el que queremos descargar datos.
- Método `_fetch_historical_temperature(start_date, end_date)`:
- Construye la petición HTTP al endpoint de archivo (`archive-api.open-meteo.com/v1/archive`) indicando latitud, longitud, rango de fechas y que queremos la variable `temperature_2m` con resolución horaria.
- Llama a la API con `requests.get(...)` y comprueba el código HTTP de respuesta. Si la respuesta es correcta, convierte la parte `hourly` del JSON en un `pandas.DataFrame` y convierte la columna `time` a `datetime` y la coloca como índice.
- Maneja errores básicos imprimiendo un mensaje si la respuesta no es 200 o si la respuesta no contiene la clave esperada.


#### Formato de salida

Devuelve un `pd.DataFrame` con al menos dos columnas: `time` (convertida y usada como índice) y `temperature_2m`. En el notebook renombramos `temperature_2m` a `temp` y construimos a partir de ahí la serie horaria que usaremos para el resto del pipeline.

In [ ]:
class TemperatureFetcher:
    def __init__(self, latitude: float, longitude: float):
        self.lat = latitude
        self.lon = longitude

    def _fetch_historical_temperature(self, start_date: datetime.date, end_date: datetime.date) -> pd.DataFrame:
        base_url = "https://archive-api.open-meteo.com/v1/archive"
        print(f"\n\tFetching weather data from Open-Meteo...", end="")
        payload = {
            "latitude": self.lat,
            "longitude": self.lon,
            "start_date": start_date.isoformat(),
            "end_date": end_date.isoformat(),
            "hourly": 'temperature_2m',
            "timezone": "auto"
        }
        response = requests.get(base_url, params=payload)
        if response.status_code == 200:
            data = response.json()
            if "hourly" in data:
                df = pd.DataFrame(data["hourly"])  # expects columns ['time', 'temperature_2m']
                df['time'] = pd.to_datetime(df['time'])
                df = df.set_index('time')
                print("DONE!")
                return df
            else:
                print("No data found in the response.")
                return pd.DataFrame()
        else:
            print(f"Request error: {response.status_code} - {response.text}")
            return pd.DataFrame()


### Descarga de datos (Murcia)

Vamos a usar el mecanismo que hemos creado para descargar la temperatura en Murcia durante los últimos 365 días.

*Cambia los valores de las variables `latitude` y `longitude` por la ubicación que te interese.*

In [ ]:
latitude = 37.99235   # Madrid
longitude = -1.13044
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=365)
fetcher = TemperatureFetcher(latitude, longitude)
df = fetcher._fetch_historical_temperature(start_date, end_date)
print(df.shape)



In [ ]:
df.head()

---

## 3) Preprocesado básico

- Nos quedamos con la columna `temperature_2m`.
- Aseguramos que es una serie horaria completa (reindexando con frecuencias horarias) y rellenamos huecos (interpolación).
- Escalamos los valores con `MinMaxScaler` para mejorar convergencia de las RNN.

In [ ]:
if 'temperature_2m' not in df.columns:
    raise ValueError('La columna temperature_2m no está presente en el DataFrame descargado.')

series = df[['temperature_2m']].rename(columns={'temperature_2m': 'temp'})
series = series.asfreq('H')
series['temp'] = series['temp'].interpolate(method='time')
scaler = MinMaxScaler()
series['temp_scaled'] = scaler.fit_transform(series[['temp']])
series.head()


In [ ]:
series['temp'].plot(figsize=(10,3));
plt.tight_layout()


En ocasiones, la API que hemos usado para descargar los datos, devuelve al final una serie de valores repetidos que debemos de borrar.

In [ ]:
series.tail()

In [ ]:
# identificar el valor de la última fila
ultima_temp= series.iloc[-1]["temp"]

# eliminar todas las filas consecutivas desde el final que tienen ese valor
while len(series) > 0 and series.iloc[-1]["temp"] == ultima_temp:
    series = series.iloc[:-1]


In [ ]:
series['temp'].plot(figsize=(10,3));
plt.tight_layout()

In [ ]:
series.tail()

#### Crear secuencias para entrenamiento (sliding window)

La función `create_sequences` crea pares `(X, y)` donde `X` es una ventana de `window_size` pasos y `y` es el valor a predecir `horizon` pasos adelante. Para la predicción un paso-ahead, `horizon=1`.

También incluimos una función para crear conjuntos preparados para aprendizaje directo multi-step (salida con `horizon` pasos), si se desea entrenar un modelo que prediga varios pasos de una sola vez.

In [ ]:
from typing import Tuple
def create_sequences(values: np.ndarray, window_size: int, horizon: int = 1) -> Tuple[np.ndarray, np.ndarray]:
    X, y = [], []
    for i in range(len(values) - window_size - (horizon - 1)):
        X.append(values[i:i + window_size])
        y.append(values[i + window_size: i + window_size + horizon])
    X = np.array(X)
    y = np.array(y)
    # reshape
    X = X.reshape((X.shape[0], X.shape[1], 1))
    if horizon == 1:
        y = y.reshape((-1,))
    return X, y

---

## 4) Particionado train/val/test


Separamos datos temporales respetando orden cronológico (no barajamos). Ajusta `train_frac` y `val_frac` según conveniencia.

In [ ]:
values = series['temp_scaled'].values
train_frac = 0.7
val_frac = 0.15
n = len(values)
train_end = int(n * train_frac)
val_end = int(n * (train_frac + val_frac))
train_vals = values[:train_end]
val_vals = values[train_end:val_end]
test_vals = values[val_end:]
print(f"Train: {len(train_vals)}, Val: {len(val_vals)}, Test: {len(test_vals)}")

---

## 5) Nuestra primera Red Neuronal Recurrente con Keras



In [ ]:
display(Image(url="https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/main/img/ml_simple_rnn.png",width=800, height=300))


Creamos una función para construir nuestra  `SimpleRNN`

In [ ]:
def build_simple_rnn(window_size: int, units: int = 32) -> tf.keras.Model:
    model = Sequential([
        SimpleRNN(units, input_shape=(window_size, 1)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
# Hiperparámetros
window_size = 24  # usar las últimas 24 horas para predecir la siguiente hora
horizon = 1
batch_size = 32
epochs = 30

# Crear secuencias
X_train, y_train = create_sequences(train_vals, window_size, horizon)
X_val, y_val = create_sequences(np.concatenate([train_vals[-window_size:], val_vals]), window_size, horizon)
X_test, y_test = create_sequences(np.concatenate([val_vals[-window_size:], test_vals]), window_size, horizon)
print(X_train.shape, y_train.shape)

es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
# Construimos nuestro modelo
rnn_model = build_simple_rnn(window_size, units=32)

In [ ]:
# Fit
history_rnn = rnn_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[es])

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(history_rnn.history['loss'], label='Entrenamiento')
plt.plot(history_rnn.history['val_loss'], label='Validación')
plt.legend()
plt.title('Error de entrenamiento y validación (MSE)')
plt.show()

### Evaluación 1-step

Vamos ahora a evaluar nuestro primer predictor usando los datos reales como inputs para cada paso (modo teacher-forcing durante la evaluación).

In [ ]:
def predict_one_step_with_truth(model, X_true):
    preds = model.predict(X_true).reshape(-1)
    return preds

In [ ]:
preds_rnn_truth = predict_one_step_with_truth(rnn_model, X_test)

Para evaluar modelos de predicción de series temporales vamos a usar dos métricas muy comunes:


- *MSE (Mean Squared Error)*: error cuadrático medio. Calcula el promedio de los errores al cuadrado, es decir:
$$MSE = \frac{1}{N} \sum_{i=1}^N (y_i - \hat{y}_i)^2$$


Penaliza más fuertemente los errores grandes al elevarlos al cuadrado.

- MAE (Error Absoluto Medio) mide la media de las diferencias absolutas entre los valores reales y las predicciones

$$MAE = \frac{1}{N} \sum_{i=1}^N (y_i - \hat{y}_i)$$


In [ ]:
def compute_metrics(y_true, y_pred, scaler):
    y_true_inv = scaler.inverse_transform(y_true.reshape(-1, 1)).reshape(-1)
    y_pred_inv = scaler.inverse_transform(y_pred.reshape(-1, 1)).reshape(-1)
    mse = mean_squared_error(y_true_inv, y_pred_inv)
    mae = mean_absolute_error(y_true_inv, y_pred_inv)
    return mse, mae

In [ ]:
mse_rnn, mae_rnn = compute_metrics(y_test, preds_rnn_truth, scaler)

print('Predicción un timestamp a la vez:')
print('RNN  MSE: {:.3f}, MAE: {:.3f}'.format(mse_rnn, mae_rnn))

In [ ]:
def mostrar_prediccion_vs_real(y_true, y_pred, scaler):
    y_true_inv = scaler.inverse_transform(y_true.reshape(-1, 1)).reshape(-1)
    y_pred_inv = scaler.inverse_transform(y_pred.reshape(-1, 1)).reshape(-1)

    plt.figure(figsize=(12,5))
    plt.plot( y_true_inv, label='Ground Truth', color='blue')
    plt.plot( y_pred_inv, label='Predicción', color='red', linestyle='--')
    plt.title('Comparación Serie Temporal: Real vs Predicha')
    plt.xlabel('Tiempo')
    plt.ylabel('Temperatura')
    plt.legend()
    plt.show()

In [ ]:
mostrar_prediccion_vs_real(y_test, preds_rnn_truth, scaler)


### *Feature engineering*

Una vez hemos realizado una primer aproximación, podemos añadir características temporales y estadísticas derivadas que suelen mejorar el rendimiento de modelos de series temporales:

- Hora del día en forma cíclica (`sin`, `cos`) para capturar la periodicidad diaria.
- Día de la semana en forma cíclica para capturar patrones semanales.
- Lags: valores pasados de la temperatura (por ejemplo, 1h, 24h).
- Estadísticas móviles: media y desviación típica de la ventana de 24h.

Además mostraremos cómo crear secuencias multivariantes para entrenar RNN/LSTM/GRU con varias features.

In [ ]:
# Añadir columnas de tiempo
series_fe = series.copy()
series_fe['hour'] = series_fe.index.hour
series_fe['dayofweek'] = series_fe.index.dayofweek

# Cíclicas para hora y día de la semana
series_fe['hour_sin'] = np.sin(2 * np.pi * series_fe['hour'] / 24)
series_fe['hour_cos'] = np.cos(2 * np.pi * series_fe['hour'] / 24)
series_fe['dow_sin'] = np.sin(2 * np.pi * series_fe['dayofweek'] / 7)
series_fe['dow_cos'] = np.cos(2 * np.pi * series_fe['dayofweek'] / 7)

# Lags (1h, 24h) sobre la temperatura original (no escalada) y luego escalamos
series_fe['lag_1'] = series_fe['temp'].shift(1)
series_fe['lag_24'] = series_fe['temp'].shift(24)

# Rolling statistics
series_fe['roll_mean_24'] = series_fe['temp'].rolling(window=24, min_periods=1).mean()
series_fe['roll_std_24'] = series_fe['temp'].rolling(window=24, min_periods=1).std().fillna(0)

# Rellenar NA que aparezcan por lag
series_fe = series_fe.fillna(method='bfill')

# Selección de features a usar
feature_cols = ['temp', 'lag_1', 'lag_24', 'roll_mean_24', 'roll_std_24', 'hour_sin', 'hour_cos', 'dow_sin', 'dow_cos']

# Escalar features
scaler_fe = MinMaxScaler()
series_fe_scaled = scaler_fe.fit_transform(series_fe[feature_cols])

# Convertir a DataFrame para mantener etiquetas
df_fe = pd.DataFrame(series_fe_scaled, index=series_fe.index, columns=feature_cols)
df_fe.head()


Actualizamos `create_sequences` para aceptar múltiples features y devolver X con shape `(n_samples, window_size, n_features)`.

In [ ]:
def create_sequences_multivariate(values: np.ndarray, window_size: int, horizon: int = 1):
    X, y = [], []
    n = values.shape[0]
    for i in range(n - window_size - (horizon - 1)):
        X.append(values[i:i + window_size])
        y.append(values[i + window_size: i + window_size + horizon, 0])  # target is la primera columna (temp)
    X = np.array(X)
    y = np.array(y)
    if horizon == 1:
        y = y.reshape((-1,))
    return X, y

In [ ]:
# Crear particiones temporales para las features
values_fe = df_fe.values
n = len(values_fe)
train_end = int(n * 0.7)
val_end = int(n * 0.85)
train_vals_fe = values_fe[:train_end]
val_vals_fe = values_fe[train_end:val_end]
test_vals_fe = values_fe[val_end:]

window_size = 24
horizon = 1
X_train_fe, y_train_fe = create_sequences_multivariate(train_vals_fe, window_size, horizon)
X_val_fe, y_val_fe = create_sequences_multivariate(np.concatenate([train_vals_fe[-window_size:], val_vals_fe]), window_size, horizon)
X_test_fe, y_test_fe = create_sequences_multivariate(np.concatenate([val_vals_fe[-window_size:], test_vals_fe]), window_size, horizon)

print('Tamaño de la serie temporal multivariante:', X_train_fe.shape, y_train_fe.shape)

Volvemos a entrenar nuestro modelo con el dataset multivariante

In [ ]:
def build_simple_rnn_fe(window_size: int, num_features: int, units: int = 32) -> tf.keras.Model:
    model = Sequential([
        SimpleRNN(units, input_shape=(window_size, num_features)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
X_train_fe.shape

In [ ]:
rnn_model_fe= build_simple_rnn_fe(window_size, X_train_fe.shape[2], units=32)

In [ ]:
history_rnn_fe = rnn_model_fe.fit(X_train_fe, y_train_fe, validation_data=(X_val_fe, y_val_fe), epochs=epochs, batch_size=batch_size, callbacks=[es])

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(history_rnn_fe.history['loss'], label='Entrenamiento')
plt.plot(history_rnn_fe.history['val_loss'], label='Validación')
plt.legend()
plt.title('Error de entrenamiento y validación (MSE)')
plt.show()


In [ ]:
preds_rnn_truth_fe = predict_one_step_with_truth(rnn_model_fe, X_test_fe)

In [ ]:
mse_rnn, mae_rnn = compute_metrics(y_test_fe, preds_rnn_truth_fe, scaler)

print('Predicción un timestamp a la vez con feature engineering:')
print('RNN(FE)  MSE: {:.3f}, MAE: {:.3f}'.format(mse_rnn, mae_rnn))

In [ ]:
mostrar_prediccion_vs_real(y_test_fe, preds_rnn_truth_fe, scaler)

### Redes LSTM

Las redes neuronales recurrentes (RNN) están diseñadas para trabajar con datos secuenciales, como series temporales o texto, ya que permiten que la información fluya de un paso temporal a otro. Sin embargo, las RNN más simples (denominadas SimpleRNN) presentan una limitación importante: tienen dificultades para aprender dependencias a largo plazo debido al problema del desvanecimiento y explosión del gradiente durante el entrenamiento. Esto provoca que la red solo recuerde información de pocos pasos anteriores, perdiendo el contexto más lejano de la secuencia.

Para resolver este problema se introdujeron las LSTM (Long Short-Term Memory), un tipo avanzado de RNN. Las LSTM incorporan una arquitectura interna más compleja con puertas (de entrada, olvido y salida) y una celda de memoria que permiten:

- Recordar información relevante durante largos intervalos de tiempo.

- Olvidar información innecesaria de manera controlada.

- Evitar el problema del desvanecimiento del gradiente, lo que mejora la estabilidad del entrenamiento.

En resumen, las LSTM mejoran a las SimpleRNN porque pueden capturar dependencias a largo plazo en las series temporales, lo que las hace más adecuadas para problemas donde el pasado lejano influye en el futuro (como predicciones de clima, lenguaje natural o secuencias financieras).

In [ ]:
display(Image(url="https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/main/img/ml_lstm.png",width=900, height=300))

In [ ]:
def build_lstm(window_size: int, units: int = 32) -> tf.keras.Model:
    model = Sequential([
        LSTM(units, input_shape=(window_size, 1)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
# Hiperparámetros
window_size = 24  # usar las últimas 24 horas para predecir la siguiente
horizon = 1
batch_size = 32
epochs = 30

# Crear secuencias (univariado)
X_train, y_train = create_sequences(train_vals, window_size, horizon)
X_val, y_val = create_sequences(np.concatenate([train_vals[-window_size:], val_vals]), window_size, horizon)
X_test, y_test = create_sequences(np.concatenate([val_vals[-window_size:], test_vals]), window_size, horizon)
print(X_train.shape, y_train.shape)

es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
# Construimos la red LSTM
lstm_model = build_lstm(window_size, units=32)

In [ ]:
# Entrenamos el modelo
history_lstm = lstm_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[es])

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(history_lstm.history['loss'], label='Entrenamiento')
plt.plot(history_lstm.history['val_loss'], label='Validación')
plt.legend()
plt.title('Error de entrenamiento y validación (MSE)')
plt.show()


In [ ]:
preds_lstm_truth = predict_one_step_with_truth(lstm_model, X_test)

In [ ]:
mse_rnn, mae_rnn = compute_metrics(y_test, preds_lstm_truth, scaler)

print('Predicción un timestamp a la vez:')
print('LSTM  MSE: {:.3f}, MAE: {:.3f}'.format(mse_rnn, mae_rnn))

In [ ]:
mostrar_prediccion_vs_real(y_test, preds_lstm_truth, scaler)


### Redes GRU

In [ ]:
def build_gru(window_size: int, units: int = 32) -> tf.keras.Model:
    model = Sequential([
        GRU(units, input_shape=(window_size, 1)),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

In [ ]:
# Crear secuencias (univariado)
X_train, y_train = create_sequences(train_vals, window_size, horizon)
X_val, y_val = create_sequences(np.concatenate([train_vals[-window_size:], val_vals]), window_size, horizon)
X_test, y_test = create_sequences(np.concatenate([val_vals[-window_size:], test_vals]), window_size, horizon)
print(X_train.shape, y_train.shape)

es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
gru_model = build_gru(window_size, units=32)
history_gru = gru_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[es])

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(history_gru.history['loss'], label='Entrenamiento')
plt.plot(history_gru.history['val_loss'], label='Validación')
plt.legend()
plt.title('Error de entrenamiento y validación (MSE)')
plt.show()

In [ ]:
preds_gru_truth = predict_one_step_with_truth(gru_model, X_test)

mse_gru, mae_gru = compute_metrics(y_test, preds_gru_truth, scaler)

print('Predicción un timestamp a la vez:')
print('GRU  MSE: {:.3f}, MAE: {:.3f}'.format(mse_gru, mae_gru))

In [ ]:
mostrar_prediccion_vs_real(y_test, preds_gru_truth, scaler)

### Prediccion autoregresiva

Vamos a evaluar otra forma de usar el predictor usando un modo autoregresivo en donde retroalimentamos las predicciones para generar el siguiente timestamp.

In [ ]:
def predict_recursive(model, seed_sequence: np.ndarray, n_steps: int) -> np.ndarray:
    window = seed_sequence.copy().reshape((window_size,))
    preds = []
    for _ in range(n_steps):
        x_in = window.reshape((1, window_size, 1))
        p = model.predict(x_in).reshape(-1)[0]
        preds.append(p)
        window = np.roll(window, -1)
        window[-1] = p
    return np.array(preds)


In [ ]:
horizon_steps = 24
seed_seq = X_test[-1].reshape((-1,))
preds_rnn_recursive = predict_recursive(rnn_model, seed_seq, horizon_steps)
preds_lstm_recursive = predict_recursive(lstm_model, seed_seq, horizon_steps)
preds_gru_recursive = predict_recursive(gru_model, seed_seq, horizon_steps)

In [ ]:
preds_rnn_recursive_inv = scaler.inverse_transform(preds_rnn_recursive.reshape(-1, 1)).reshape(-1)
preds_lstm_recursive_inv = scaler.inverse_transform(preds_lstm_recursive.reshape(-1, 1)).reshape(-1)
preds_gru_recursive_inv = scaler.inverse_transform(preds_gru_recursive.reshape(-1, 1)).reshape(-1)

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(preds_rnn_recursive_inv, label='RNN recursivo')
plt.plot(preds_lstm_recursive_inv, label='LSTM recursivo')
plt.plot(preds_gru_recursive_inv, label='GRU recursivo')
plt.legend()
plt.title('Predicciones autoregresivas (24 horas a partir de la última ventana de test)')
plt.show()

## Conclusiones: diferencias entre RNN, LSTM y GRU

- **RNN simple (SimpleRNN):** unidad recurrente básica. Mantiene un estado oculto que se actualiza cada paso. Sufre del problema de *vanishing gradients* para dependencias a largo plazo; suele ser rápido y simple, apropiado para relaciones a corto plazo.
- **LSTM (Long Short-Term Memory):** introduce una celda de memoria y puertas (input, forget, output) que controlan el flujo de información. Diseñado para aprender dependencias a largo plazo y evitar el *vanishing gradient*.
- **GRU (Gated Recurrent Unit):** versión simplificada de LSTM con menos puertas (update/reset). A menudo alcanza rendimientos similares a LSTM con menos parámetros y entrenamiento más rápido.


In [ ]:
display(Image(url="https://raw.githubusercontent.com/fterroso/curso_ia_smart_cities/main/img/ml_rnn_lstm_gru.png",width=900, height=400))

---

## Ejercicios propuestos

1. Experimenta con el `window_size`.
2. Horizon: prueba `horizon=1` vs `horizon=24`
3. Features adicionales: añade variables como hora del día, día de la semana, variables meteorológicas auxiliares.
4. Escalado: probar `StandardScaler` frente a `MinMaxScaler`.


¡Eso es todo amigos!